In [ ]:
// ---------- Part 1/6 ----------
#include <SPI.h>
#include <MFRC522.h>
#include <Keypad.h>
#include <Wire.h>
#include <LiquidCrystal_I2C.h>
#include <Servo.h>

// RFID, LCD, Servo, Pins
#define RST_PIN 9
#define SS_PIN 10
#define STAND_SENSOR 0   // D0
#define SEAT_SENSOR_1 8  // D1
#define SEAT_SENSOR_2 2  //
#define SERVO_PIN 3


MFRC522 mfrc522(SS_PIN, RST_PIN);
LiquidCrystal_I2C lcd(0x27, 20, 4);
Servo gateServo;

// UID, User Data
struct User {
  String name;
  int balance;
  bool initialized;
};

User users[] = {
  {"Adnan", 1000, false},    // UID: 4A183203
  {"Moona", 1000, false},    // UID: 33EE9A14
  {"Sakib", 1000, false},    // UID: FDABC301
  {"Ethika", 20, false}      // UID: 93C441C5
};

String uids[] = {
  "4A183203", "33EE9A14", "FDABC301", "93C441C5"
};

User* currentUser = nullptr;
String currentUID = "";

// Keypad Setup
const byte ROWS = 4;
const byte COLS = 4;
char keys[ROWS][COLS] = {
  {'D','C','B','A'},
  {'#','9','6','3'},
  {'0','8','5','2'},
  {'*','7','4','1'}
};
byte rowPins[ROWS] = {A0, A1, A2, A3};
byte colPins[COLS] = {4, 5, 6, 7};
Keypad keypad = Keypad(makeKeymap(keys), rowPins, colPins, ROWS, COLS);

void setup() {
  Serial.begin(9600);
  Wire.begin();
  SPI.begin();
  mfrc522.PCD_Init();

  pinMode(STAND_SENSOR, INPUT);
  pinMode(SEAT_SENSOR_1, INPUT);
  pinMode(SEAT_SENSOR_2, INPUT);


  gateServo.attach(SERVO_PIN);
  gateServo.write(0);

  lcd.init();
  lcd.backlight();
  lcd.setCursor(0, 0);
  lcd.print("System Starting...");
  delay(2000);
  lcd.clear();
}
// ---------- Part 2/6 ----------
void displaySeatStatus() {
  lcd.clear();
  lcd.setCursor(0, 0);
  lcd.print("Seat1:"); lcd.print(digitalRead(SEAT_SENSOR_1) ? "1" : "O");
  lcd.setCursor(9, 0);
  lcd.print("Seat2:"); lcd.print(digitalRead(SEAT_SENSOR_2) ? "1" : "O");
  lcd.setCursor(0, 1);
  lcd.print("Stand :"); lcd.print(digitalRead(STAND_SENSOR) ? "1" : "O");
  lcd.setCursor(0, 2);
  lcd.print("Scan your card...");
}



void waitForCard() {
  while (true) {
    displaySeatStatus();
    if (mfrc522.PICC_IsNewCardPresent() && mfrc522.PICC_ReadCardSerial()) {
      currentUID = "";
      for (byte i = 0; i < mfrc522.uid.size; i++) {
        currentUID += String(mfrc522.uid.uidByte[i] < 0x10 ? "0" : "");
        currentUID += String(mfrc522.uid.uidByte[i], HEX);
      }
      currentUID.toUpperCase();
      mfrc522.PICC_HaltA();
      break;
    }
    if (keypad.getKey() == '#') toggleGate();
    delay(200);
  }
}

void assignUser() {
  currentUser = nullptr;
  for (int i = 0; i < 4; i++) {
    if (currentUID == uids[i]) {
      currentUser = &users[i];
      if (!currentUser->initialized) {
        currentUser->initialized = true;  // Assign balance only once
      }
      break;
    }
  }

  lcd.clear();
  if (currentUser != nullptr) {
    lcd.setCursor(0, 0);
    lcd.print("Welcome ");
    lcd.print(currentUser->name);
    delay(2000);
  } else {
    lcd.setCursor(0, 0);
    lcd.print("Invalid Card!");
    delay(2000);
  }
}
// ---------- Part 3/6 ----------
int selectDestination(char &destKey, int &sitFare, int &standFare) {
  lcd.clear();
  lcd.setCursor(0, 0);
  lcd.print("Where to go?");
  lcd.setCursor(0, 1);
  lcd.print("A.Agargoan B.J.Gate");
  lcd.setCursor(0, 2);
  lcd.print("C.Mhkli D.Gulshan");

  while (true) {
    destKey = keypad.getKey();
    if (destKey == '#' || destKey == '*') toggleGate();

    if (destKey == 'A') { sitFare = 60; standFare = 30; return 1; }
    if (destKey == 'B') { sitFare = 40; standFare = 20; return 1; }
    if (destKey == 'C') { sitFare = 20; standFare = 10; return 1; }
    if (destKey == 'D') { sitFare = 10; standFare = 5; return 1; }

    delay(100);
  }
}

int chooseTravelMode(int sitFare, int standFare) {
  lcd.clear();
  lcd.setCursor(0, 0);
  lcd.print("A. Sit "); lcd.print(sitFare); lcd.print(" tk");
  lcd.setCursor(0, 1);
  lcd.print("B. Stand "); lcd.print(standFare); lcd.print(" tk");

  char key = 0;
  while (true) {
    key = keypad.getKey();
    if (key == 'A') return 1;
    if (key == 'B') return 2;
    if (key == '#' || key == '*') toggleGate();
    delay(100);
  }
}

bool checkAvailability(int mode) {
  if (mode == 1) {
    if (digitalRead(SEAT_SENSOR_1) == 1 || digitalRead(SEAT_SENSOR_2) == 1) return true;
    lcd.clear();
    lcd.print("NO SEATS Available");
    delay(2000);
    currentUser = nullptr;
    return false;

  } else {
    if (digitalRead(STAND_SENSOR) == 1) return true;
    lcd.clear();
    lcd.print("NO STANDING SPACE");
    delay(2000);
    currentUser = nullptr;
    return false;

  }

}
// ---------- Part 4/6 ----------
bool processFare(int mode, int sitFare, int standFare) {
  int fare = (mode == 1) ? sitFare : standFare;

  if (currentUser->balance >= fare) {
    currentUser->balance -= fare;
    lcd.clear();
    lcd.setCursor(0, 0);
    lcd.print("Fare: "); lcd.print(fare);
    lcd.setCursor(0, 1);
    lcd.print("Remaining: "); lcd.print(currentUser->balance);
    delay(2000);
    return true;
  } else {
    lcd.clear();
    lcd.setCursor(0, 0);
    lcd.print("Insufficient Fund");
    lcd.setCursor(0, 1);
    lcd.print("Press * to Recharge");
    lcd.setCursor(0, 2);
    lcd.print("Press 1 to Exit");

    while (true) {
      char key = keypad.getKey();
      if (key == '*') {
        doRecharge();  // Now auto-returns to waitForCard
        return false;
      }
      if (key == '1') {
        lcd.clear();
        lcd.print("Transaction Cancelled");
        delay(2000);
        return false;
      }
      if (key == '#') toggleGate();
      delay(100);
    }
  }
}

void doRecharge() {
  lcd.clear();
  lcd.setCursor(0, 0);
  lcd.print("Enter amount:");
  String amountStr = "";
  char key;

  while (true) {
    key = keypad.getKey();
    if (key >= '0' && key <= '9') {
      amountStr += key;
      lcd.setCursor(0, 1);
      lcd.print("Recharge: ");
      lcd.print(amountStr);
    } else if (key == '*') {
      int amt = amountStr.toInt();
      currentUser->balance += amt;
      lcd.clear();
      lcd.setCursor(0, 0);
      lcd.print("Recharged: ");
      lcd.print(amt);
      lcd.setCursor(0, 1);
      lcd.print("Balance: ");
      lcd.print(currentUser->balance);
      delay(3000);
      lcd.clear();
      waitForCard();
      return;
    }
    if (key == '#') toggleGate();
    delay(100);
  }
}
// ---------- Part 5/6 ----------
void promptToProceed() {
  lcd.clear();
  lcd.setCursor(0, 0);
  lcd.print("To Recharge: *");
  lcd.setCursor(0, 1);
  lcd.print("To Go On: 0");

  while (true) {
    char key = keypad.getKey();
    if (key == '*') {
      doRecharge(); // This will reset back to RFID scan
      return;
    }
    if (key == '0') {
      openGate();
      return;
    }
    if (key == '#') toggleGate();
    delay(100);
  }
}

void openGate() {
  lcd.clear();
  lcd.setCursor(0, 0);
  lcd.print("Opening Gate...");

  gateServo.write(90);
  delay(10000);  // Gate stays open 10s
  gateServo.write(0);

  lcd.clear();
  lcd.print("Thank you!");
  delay(2000);
}

void toggleGate() {
  static bool gateOpen = false;
  gateOpen = !gateOpen;

  if (gateOpen) {
    lcd.clear();
    lcd.print("Manual Gate OPEN");

    gateServo.write(90);
  } else {
    lcd.clear();
    lcd.print("Manual Gate CLOSED");

    gateServo.write(0);
  }
  delay(1500);
}
// ---------- Part 6/6 ----------
void loop() {
  // Step A: Show seat/stand status and wait for card
  waitForCard();
  assignUser();

  if (currentUser == nullptr) return;

  // Step B: Destination & fare
  char destKey;
  int sitFare, standFare;

  if (!selectDestination(destKey, sitFare, standFare)) return;

  // Step C: Travel mode choice
  int travelMode = chooseTravelMode(sitFare, standFare);

  // Step D: Check seat/stand availability
  if (!checkAvailability(travelMode)) {
    currentUser = nullptr;  // Fix: Reset after failure
    return;
  }

  // Step E: Process fare and show balance
  if (!processFare(travelMode, sitFare, standFare)) {
    currentUser = nullptr;  // Fix: Reset after failure
    return;
  }

  // Step F: Option to recharge or proceed
  promptToProceed();

  // Step G: Return to main loop
  lcd.clear();
  currentUser = nullptr;  //  Always reset for next user
}